In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
import os

from langchain_neo4j import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://127.0.0.1:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "12345678"

graph = Neo4jGraph(refresh_schema=False)

In [3]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

llm_transformer = LLMGraphTransformer(llm=llm)

In [4]:
# import json
# with open('output.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)

# # Nếu là list gồm dict có khóa 'text', nối lại thành một đoạn văn
# if isinstance(data, list):
#     text = '\n'.join(item['text'] for item in data if 'text' in item)
# else:
#     text = str(data)

In [5]:
# from langchain_core.documents import Document

# documents = [Document(page_content=text)]
# graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)
# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")

In [6]:
# # 1. Lấy node.name đã tồn tại
# existing_nodes = graph.query("MATCH (n) WHERE exists(n.id) RETURN DISTINCT n.id AS id")
# existing_names = {row["id"] for row in existing_nodes}

# # 2. Lọc graph_documents
# filtered_documents = []

# for doc in graph_documents:
#     unique_nodes = [
#         node for node in doc.nodes
#         if node.id not in existing_names and node.id.strip() != ""
#     ]
#     if unique_nodes:
#         doc.nodes = unique_nodes
#         filtered_documents.append(doc)

# # 3. Thêm vào graph
# graph.add_graph_documents(filtered_documents)

In [15]:
import json
from langchain_core.documents import Document

# --- BƯỚC 1: Đọc file output.json ---
with open('output.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# --- BƯỚC 2: Lấy danh sách id đã tồn tại ---
existing_nodes = graph.query("MATCH (n) WHERE n.id IS NOT NULL RETURN DISTINCT n.id AS id")
existing_ids = {row["id"] for row in existing_nodes}

total_new_nodes = 0

# --- BƯỚC 3: Chạy từng batch ---
for i in range(0, len(data), 5):
    batch = data[i:i+5]
    batch_text = '\n'.join(item.get('content', '') for item in batch if 'content' in item)
    documents = [Document(page_content=batch_text)]

    try:
        graph_docs = await llm_transformer.aconvert_to_graph_documents(documents)

        # --- Lọc trùng node trong batch này ---
        filtered_documents = []
        for doc in graph_docs:
            unique_nodes = [
                node for node in doc.nodes
                if node.id not in existing_ids and node.id.strip() != ""
            ]
            if unique_nodes:
                doc.nodes = unique_nodes
                filtered_documents.append(doc)

                # Cập nhật existing_ids để không thêm trùng ở batch sau
                existing_ids.update(node.id for node in unique_nodes)

        # --- Thêm vào graph nếu có node mới ---
        if filtered_documents:
            graph.add_graph_documents(filtered_documents)
            print(f"✅ Added batch {i//5 + 1} with {len(filtered_documents[0].nodes)} new nodes.")
            total_new_nodes += len(filtered_documents[0].nodes)
        else:
            print(f"⚠️ Skipped batch {i//5 + 1} (no new nodes).")

    except Exception as e:
        print(f"❌ Error at batch {i//5 + 1}: {e}")

print(f"\n🎉 Tổng số node mới được thêm: {total_new_nodes}")


✅ Added batch 1 with 16 new nodes.
✅ Added batch 2 with 20 new nodes.
✅ Added batch 3 with 18 new nodes.
✅ Added batch 4 with 23 new nodes.
✅ Added batch 5 with 18 new nodes.
✅ Added batch 6 with 24 new nodes.
✅ Added batch 7 with 16 new nodes.
✅ Added batch 8 with 30 new nodes.
✅ Added batch 9 with 15 new nodes.
✅ Added batch 10 with 15 new nodes.
✅ Added batch 11 with 27 new nodes.
✅ Added batch 12 with 15 new nodes.
✅ Added batch 13 with 15 new nodes.
✅ Added batch 14 with 13 new nodes.
✅ Added batch 15 with 12 new nodes.
✅ Added batch 16 with 17 new nodes.
✅ Added batch 17 with 23 new nodes.
✅ Added batch 18 with 17 new nodes.
✅ Added batch 19 with 23 new nodes.
✅ Added batch 20 with 13 new nodes.
✅ Added batch 21 with 29 new nodes.
✅ Added batch 22 with 20 new nodes.
✅ Added batch 23 with 7 new nodes.
✅ Added batch 24 with 10 new nodes.
✅ Added batch 25 with 15 new nodes.
✅ Added batch 26 with 32 new nodes.
✅ Added batch 27 with 12 new nodes.
✅ Added batch 28 with 9 new nodes.
✅ A

CancelledError: 

In [14]:
graph.query("MATCH (n) DETACH DELETE n")

[]